# Detecting Newborn Screening Disorders with WGS

Illumina variants were annotated with the NBS gene table provided by Ben Solomon at ITMI. Variants located in NBS gene regions were annotated with Kaviar frequency, ClinVar clinical significance ratings, DANN predicted pathogenicity scores for SNV's, and coding consequence predictions using SnpEff.

Variants were filtered for appropriate mode of inheritance and then labeled as predictive when they contain the presence of mutation(s) with appropriate inheritance (eg, 2 bi-allelic pathogenic mutations for a recessive disorder).  

Mutations defined strictly as either:  
- Annotated in ClinVar with a clinical significance of 4 or 5, but never 2 or 3  

OR  
- Novel (in Kaviar with frequency lower than specified below, or not listed in Kaviar) and predicted to be disease-causing with a SnpEff impact score of 'high'

## User Specified Parameters

In [16]:
import time

# enter kaviar frequency threshold
kav_freq = '.01'

# enter variant quality score cutoff
qual_cutoff = '30'

# enter minimum read depth
read_depth = '10'

# enter dictionary of samples to examine in format 'sample_id':['family_id','member_id']
sample_list = 'all'

# enter as NB, M, and/or F, or 'all' to include extended family if available
trio_member = 'NB','M','F'

# enter user database to output tables
out_db = 'users_selasady'

# enter database.name for variant table
variant_table = 'p7_platform.brady_variant'

# enter database.name for global variants table
gvt = 'p7_ref_grch37.global_variants'

# enter database.name for coding consequences table
coding_table = 'p7_ref_grch37.coding_consequences'

# name for output files
out_name = 'nbs_' + time.strftime("%y%m%d")

## Transform of NBS Gene Table

The NBS Gene Table provided by ITMI was transformed as follows and uploaded to impala:   

- A 'level' column was added to represent the color coding found in the file, with the following options: red, yellow, null  
- All commas were replaced with colons to prevent errors while importing csv file  
- All semicolons were replaced with colons for consistency of data  
- Spaces in column names were replaced with underscores  
- Special characters were removed from column names

## Adding positional information to the NBS gene list with UCSC

Genomic position, gene and strand were added to the NBS Gene list by joining it with the ucsc_knowngenes table using gene name. 

        CREATE TABLE acmg_ucsc AS (
            SELECT acmg.*, ucsc.chrom, ucsc.txstart, ucsc.txstop
            FROM acmg_genes as acmg, ucsc_genes as ucsc
            WHERE acmg.gene = ucsc.gene_name
)

## Connect to impala with impyla

In [11]:
# import needed modules
import sys
sys.path.append('/Users/selasady/impala_scripts/parse_impala/')
from impala.util import as_pandas
import pandas as pd
from impala.dbapi import connect
import parse_module as pi

# disable extraneous pandas warning
pd.options.mode.chained_assignment = None

# to connect to specific database, use the database argument
conn=connect(host='glados19', port=21050)

## Locate High Risk Variants in NBS Gene Regions

Variants were filtered to include only variants that passed the following parameters: 
    - fully called genotype
    - read depth >= 10
    - quality score >= 30
    
These filtered variants were then joined with the Global Vriants table and Coding Consequences table to filter for variants with the following parameters: 
    - Marked as clinicall significant in ClinVar
    - Or marked as rare in Kaviar with a high impact coding consequence

### Parse User Arguments

In [13]:
# parse trio arg
member_arg = pi.label_member('bv', trio_member)

# parse sample id argument
subject_list = pi.select_samples('bv', sample_list)

# list of user args to join
arg_list = [subject_list, member_arg]
sample_args = pi.join_args(arg_list)

print 'The following argument will be added to the query: \n'
print sample_args

The following argument will be added to the query: 

 AND (bv.sample_id LIKE '%03' OR bv.sample_id LIKE '%01' OR bv.sample_id LIKE '%02')


### Query variant table on impala

Filtered variants will be joined with the global variants table and coding consequences table to filter for variants with the following parameters: 
    - ClinVar significance rating
    - 

In [ ]:
nbs_vars = '''
WITH vars AS (
    SELECT bv.sample_id, bv.chrom, nbs.txstart + 1000 as  start_1000, nbs.txstop + 1000 as stop_1000, bv.pos, bv.ref, bv.alt, bv.id as rs_id, bv.qual, bv.filter,
        regexp_replace(bv.gt, '1/2', '0/1') as geno, nbs.gene, nbs.condition, nbs.va_name, nbs.inheritance,
        nbs.allelic_condition, nbs.in_pgb, nbs.mrna
    FROM {} bv, {} nbs
    WHERE bv.qual >= 30
    AND bv.dp >= 10
    AND bv.gt IS NOT NULL
    AND bv.chrom = nbs.chrom
    AND bv.pos BETWEEN nbs.txstart + 1000 and nbs.txstop + 1000
   )
    
    SELECT vars.*, gv.kav_freq, gv.clin_sig, gv.clin_dbn, gv.rs_id,
        gv.dann_score, cd.effect, cd.impact, cd.feature, cd.feature_id, 
        cd.biotype, cd.rank, cd.hgvs_c, cd.hgvs_p
        FROM vars
        JOIN {} gv
            ON vars.chrom = gv.chrom
            AND vars.pos = gv.pos
            AND vars.ref = gv.ref
            AND vars.alt = gv.alt
        JOIN {} cd
            ON vars.chrom = cd.chrom
            AND vars.pos = cd.pos
            AND vars.ref = cd.ref
            AND vars.alt = cd.alt  
        WHERE (((gv.kav_freq < .01 or gv.kav_freq IS NULL) AND cd.impact = 'HIGH') OR
        ((clin.clin_sig NOT REGEXP '3|2[^5]|2$' AND  clin.clin_sig REGEXP '4|[^25]5|^5'))
         )
'''

# run kaviar annotation query
nbs_df = run_query(nbs_vars, 'nbs_variants')

## Filter for proper mode of inheritance

The dataframe was separated into mother, father and newborn predictive het variants to search for compound hertozygosity.

In [68]:
mom_hets, dad_hets, nb_het1, nb_het2, nb_het3 = pimp.find_AR_cands(nbs_annot)

### Finding Dominant and Homozygous Recessive Disorders

The newborn and parent variants were subset to report:  
- All variants in regions of dominant disorders  
- All homozygous recessive variants in regions of autosomal recessive disorders

In [72]:
nb_dominant = pimp.find_dominant_nb(nbs_annot)

nb_hom_recessive = pimp.find_hom_rec(nbs_annot)

### Looking for compunt heterozygosity

Newborn heterozygous variants were subset to look for compound heterozygosity. This analysis was only performed on newborns where both parents were also sequenced.

In [114]:
# function to find matching parent variants
def find_parent_vars(nb_df, parent_df):
    # merge dataframes on by variant position
    merged_df = pd.merge(nb_df, parent_df, on=['chrom', 'pos', 'ref', 'alt'], how='inner')
    # rename parent sample_id column to avoid dropping when removing '_y' cols
    merged_df.rename(columns = {'member_y':'from_parent'}, inplace=True)
    # drop extra y columns from merge with fathers
    drop_y(merged_df)
    #remove _x from colnames
    merged_df.rename(columns=lambda x: x.replace('_x', ''), inplace=True)
    return merged_df
    
# run function for each group of newborns
def match_parents(nb_df):
    if (len(mom_hets) > 0) and (len(dad_hets) > 0):
        nb_and_mom = find_parent_vars(nb_df, mom_hets)
        nb_and_dad = find_parent_vars(nb_df, dad_hets)
        # merge variants found in either mother or father
        het_cands = pd.concat([nb_and_mom,nb_and_dad]).drop_duplicates().reset_index(drop=True)
        # group variants by gene name
        by_gene = het_cands.groupby(['gene_name', 'family_id'])
        return by_gene
    else:
        print "No compound het variants"
        
# run function for each group of newborns
het1_grouped = match_parents(nb_het1)
het2_grouped = match_parents(nb_het2)
het3_grouped = match_parents(nb_het3)

After grouping the variants by gene and family, the variants will be filtered to keep only variants with at least one different variant coming from the mother and one from the father.

In [115]:
# function to find compound hets
def find_comphets(gene_group, comphet_list_name):
    for name, group in gene_group:
        # if there is a variant in more than one position
        if group.pos.nunique() > 1:
            # and there are more than one variants from both parents
            if len(group[group['from_parent'] == 'M'] > 1) and len(group[group['from_parent'] == 'F'] > 1):
                comphet_list_name.append(group)
            # or if there is only one variant from each parent
            elif len(group[group['from_parent'] == 'M'] == 1) and len(group[group['from_parent'] == 'F'] == 1):
                # and those variants are different
                if len(group[group['from_parent'] == 'M'].pos - group[group['from_parent'] == 'F']) > 0:
                        comphet_list_name.append(group)

# create empty list to store comp_hets
comp_hets = []

het1_df = find_comphets(het1_grouped, comp_hets)     
het2_df = find_comphets(het2_grouped, comp_hets)
het3_df = find_comphets(het3_grouped, comp_hets)

# check if comphets found
def comphet_check(df):
    if df:
        comp_hets.append(df)

# check for each nb df
comphet_check(het1_df)
comphet_check(het2_df)
comphet_check(het3_df)

# if any comp  hets were found, append to list and convert to df
if len(comp_hets) > 0:
    comphet_df = pd.concat(comp_hets)
    comphet_df.name = 'comp_het'
else:
    print 'No comp hets found.'
    comphet_df = pd.DataFrame()
    comphet_df.name = 'comp_het'

## QA

In [154]:
review_these = []

# make sure all dominant variants are inherited as AD
if len(nb_dominant[(~nb_dominant['inheritance'].isin(['AD']))]) > 0:
     review_these.append(group)

# make sure all homozygous recessive are homozygous alt and AR
if len(nb_hom_recessive[(~nb_hom_recessive['inheritance'].isin(['AR'])) & (nb_hom_recessive['gt'] != '1/1')]) > 0:
    review_these.append(group)

# make sure all compound hets are het and AR 
if len(comphet_df[(~comphet_df['inheritance'].isin(['AR'])) | (comphet_df['gt'] != '0/1')]) > 0:
     review_these.append(group)

# check that there is more than one variant per gene
het_groupy = comphet_df.groupby(['gene'])

for name, group in het_groupy:
        # check that there is a variant in more than one position
        if group.pos.nunique() < 1:
            review_these.append(group)
        # check that there is at least one variant from the mother
        if len(group[group['from_parent'] == 'M']) == 0:
            review_these.append(group)
        # check that there is at least one variant from the father
        if len(group[group['from_parent'] == 'F']) == 0:
            review_these.append(group)
        # check that there is at least one different variant from each parent
        if len(group[(group['from_parent'] == 'M')].pos - group[(group['from_parent'] == 'F')].pos)  == 0:
            review_these.append(group)

## Results

In [ ]:
reload(pimp)

In [117]:
# report variant counts
def report_result_counts(results_df):
    if len(results_df) > 0:
        print str(len(results_df)) + ' {} variants found.'.format(results_df.name)
        condition = results_df['condition'].value_counts()
        genes = results_df['gene'].unique()
        print 'Condition: \n', condition, '\n'
        print 'Affected gene(s): \n', genes, '\n'
    else:
         print "No {} variants found. \n".format(results_df.name)
        
print "Variants found:"
report_result_counts(nb_dominant)
report_result_counts(nb_hom_recessive)
report_result_counts(comphet_df)
print "\n"

Variants found:
28 dominant variants found.
Condition: 
Hypothyroidism: congenital: nongoitrous 2    16
Pallister-Hall syndrome 2                    12
dtype: int64 

Affected gene(s): 
['PAX8' 'GLI2'] 

580 hom_recessive variants found.
Condition: 
Hyperprolinemia: type I                                         400
Severe combined immunodeficiency: autosomal recessive: T-cell negative: B-cell positive: NK cell positive     60
Adrenal hyperplasia: congenital: due to 21-hydroxylase deficiency: Hyperandrogenism: nonclassic type: due to 21-hydroxylase deficiency     60
Tyrosinemia: type III                                            32
Maple syrup urine disease: type II                               18
Severe combined immunodeficiency                                 10
dtype: int64 

Affected gene(s): 
['IL7R' 'HPD' 'PRODH' 'CYP21A2' 'MTHFD1' 'DBT'] 

251 comp_het variants found.
Condition: 
Severe combined immunodeficiency: autosomal recessive: T-cell negative: B-cell positive: NK cell p

## Saving Output

In [155]:
# add from_parent column to dom and hom_rec so we can keep info for comp_het
nb_dominant['from_parent'] = 'NA'
nb_hom_recessive['from_parent'] = 'NA'

# merge and mark predicted variants
def merge_predictors(df, out_list):
    if len(df) > 0:
        df['var_type'] = df.name
        out_list.append(df)
        print "Saving {} {} variants to current working directory".format(len(df), df.name)
    else:
        print "No {} variants to save.".format(df.name)

# list to store patho output
predict_list = []            

merge_predictors(nb_dominant, predict_list)
merge_predictors(nb_hom_recessive, predict_list)
merge_predictors(comphet_df, predict_list)

# merge results
merged_patho = pd.concat(predict_list, axis=0)

# put the columns back in order because pandas
cols = ['sample_id', 'chrom', 'pos', 'id', 'ref', 'alt', 'gt', 'gene',  'gene_id', 'allele_freq', 
        'condition','inheritance', 'clin_sig', 'clin_dbn', 'transcript_id', 'strand',   
        'impact', 'effect', 'dbnfsp_predicted', 'dann_score', 'cadd_raw', 'qual', 'filter', 
        'clin_hgvs', 'omim_phenotype', 'va_name', 'allelic_conditions', 'comments', 'level',
        'aa_alt', 'sift_score', 'sift_pred', 'polyphen2_hdiv_score', 'polyphen2_hdiv_pred', 
        'polyphen2_hvar_score', 'polyphen2_hvar_pred', 'fathmm_score', 'fathmm_pred',  
        'mutation_taster_pred', 'mutation_assessor_score', 'mutation_assessor_pred', 'provean_score', 'provean_pred', 
        'interpro_domain', 'exac_af', 'rank', 'hgvs_c', 'hgvs_p', 'from_parent', 'var_type',   
        'member','family_id', 'feature_type', 'gene_name','id']

merged_out = merged_patho[cols]

# remove unnecessary columns
merged_out.drop(merged_out.columns[[50, 51, 52, 52,53,54]], axis=1, inplace=True)

# save to file
merged_out.to_csv('predicted_NBS_{}.tsv'.format(time.strftime("%y%m%d")), sep= '\t', header=True, encoding='utf-8', \
                                                                                                    index=False)

if len(review_these) > 0:
    print 'Some variants were flagged for review and saved to nbs_{)_review.tsv'.format(time.strftime("%y%m%d"))
    for_review = pd.DataFrame(review_these)
    for_review.to_csv('nbs_{)_review.tsv'.format(time.strftime("%y%m%d")), sep='\t', header=True, encoding='utf-8', 
                                                 index=False)

Saving 28 dominant variants to current working directory
Saving 580 hom_recessive variants to current working directory
Saving 251 comp_het variants to current working directory


## To Do

- make package
- change location of tab2vcf.py function according to location in package
- report only nb vars
- mark with variant type
- why is sample id _22 showing up? 